In [42]:
import pandas as pd
import numpy as np
import tqdm
import re
import scipy.stats as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [43]:
dataset = pd.read_csv("letras_final.csv", parse_dates=["date"])
dataset.drop(['Unnamed: 0', 'Unnamed: 0.1' ], axis = 1, inplace = True)
dataset.isna().sum()

artist_name           0
title                 0
album                 0
featured_artists      0
lyrics               58
date                276
url                   0
dtype: int64

In [44]:
dataset["artist_name"].value_counts()

Phoenix RDC              69
Sam The Kid              59
ProfJam                  49
Boss AC                  48
Xeg                      44
Bispo                    44
Jimmy P                  43
Regula                   41
Dealema                  39
Allen Halloween          38
LON3R JOHNY              37
Da Weasel                37
Dillaz                   36
Valete                   35
NGA                      35
Piruka                   31
GROGNation               31
Prodígio                 30
Alcool Club              30
Nerve                    29
Agir                     27
Slow J                   26
Conjunto Corona          26
Keso                     25
Plutonio                 22
Deejay Telio             20
Kappa Jotta              20
X-Tense                  18
Tóy Tóy T-Rex            18
Força Suprema            17
Wet Bed Gang             17
Carlão (PacMan)          16
Mike El Nite             15
Chico da Tina            15
Cálculo                  15
Estraca             

In [45]:
dataset['lyrics'] = dataset['lyrics'].fillna(value='')

def remove_punct(s):
    new_lyr = re.sub(r'[^\w\s]', '', s).lower()
    new_lyr = re.sub(r'[\n]', ' ', new_lyr).lower()
    return new_lyr

dataset.lyrics = dataset.lyrics.apply(remove_punct)

In [46]:
#lyrics length
dataset["lyrics_len"] = [len(x) for x in dataset["lyrics"]]

In [47]:
# remover stopwords e tokenizar

stopwords_pt = nltk.corpus.stopwords.words('portuguese')
stopwords_pt.append("porque")
dataset['no_stopwords'] = dataset.apply(lambda x : ' '.join([word for word in x['lyrics'].split(' ') if word not in stopwords_pt]), axis = 1)
dataset['tokens'] = dataset['no_stopwords'].apply(nltk.word_tokenize)

In [48]:
dicionario = pd.read_csv("dicionario_limpo.csv")
dicionario

,Unnamed: 0,term,description,level,synonym,scale
0,1,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
1,4,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...,1
2,11,à beça,em grande quantidade,coloquial,NaN,1
3,13,abichanado,&bullet; homossexual masculino passivo,calão,arrombado; paneleiro; larilas; lelé; bicha; b...,2
4,20,abono de família,"órgão sexual masculino, pénis, pênis",coloquial,NaN,1
...,...,...,...,...,...,...
715,6508,xarro,NaN,coloquial,cavalo; heroína; charro,1
716,6511,xibungo,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
717,6512,xilindró,cadeia,coloquial,pildra; choça; prisão; prisa; calabouço; masm...,1
718,6518,xulo,é um:,calão carroceiro,none,3


In [49]:
#finding matches for swear words

lyrics_sets = dataset["tokens"]
swear_words = list(dicionario["term"])
scores = pd.Series(dicionario.scale.values,index=dicionario.term).to_dict()
score_data = []
swear_words_by_song = []

for lyrics in lyrics_sets:
  total_swear_score = 0
  swear_words_in_lyrics = [token for token in lyrics if token in swear_words]

  for swear in swear_words_in_lyrics:
      total_swear_score += scores.get(swear)

  swear_words_by_song.append(swear_words_in_lyrics)
  score_data.append(total_swear_score)

dataset["swears"] = swear_words_by_song
dataset["score"] = score_data

In [50]:
mean_by_artist = dataset.groupby('artist_name')['score'].mean()
mean_by_artist

artist_name
5-30                      6.000000
9 Miller                 18.100000
Agir                      2.037037
Alcool Club               6.466667
Allen Halloween          12.131579
Bispo                     9.727273
Boss AC                   3.750000
Carlão (PacMan)           4.750000
Chico da Tina            12.266667
Classe Crua               5.090909
Conjunto Corona           5.192308
Cálculo                   4.800000
Da Weasel                 4.513514
Dealema                   5.871795
Deejay Telio             11.750000
Deezy (Dope Muzik)        5.333333
Dillaz                    6.527778
Estraca                   4.142857
Força Suprema             8.941176
Fínix MG                  9.769231
GROGNation                8.129032
Gson                      2.000000
Holly Hood                8.777778
Jimmy P                   3.302326
Kappa Jotta               8.650000
Keso                      5.960000
LON3R JOHNY               2.648649
Mike El Nite              6.933333
NGA     

In [51]:
#filtering misogynistic terms

lady_parts = dicionario[dicionario['description'].str.contains('órgão sexual feminino', na=False)]
men_parts = dicionario[dicionario['description'].str.contains('órgão sexual masculino', na=False)]
women_rude = dicionario[dicionario['synonym'].str.contains('prostituta', na=False)]
oral_sex = dicionario[dicionario['description'].str.contains('sexo oral', na=False)]
anal_sex = dicionario[dicionario['description'].str.contains('sexo anal', na=False)]
semen = dicionario[dicionario['description'].str.contains('esperma', na=False)]

dicionario_misoginia = pd.concat([lady_parts, women_rude, men_parts, semen, oral_sex], ignore_index= True)
dicionario_misoginia

,Unnamed: 0,term,description,level,synonym,scale
0,525,boceta,órgão sexual feminino,calão carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,3
1,977,cona,órgão sexual feminino,calão muito carroceiro,pipi; pito; pitaço; pirona; rata; vagina; nin...,4
2,978,conaça,órgão sexual feminino,calão muito carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,4
3,1106,crica,órgão sexual feminino,calão muito carroceiro,cona; pipi; pito; pitaço; pirona; rata; vagin...,4
4,1957,entrefolhos,órgão sexual feminino,calão,cona; pipi; pito; pitaço; pirona; rata; vagin...,2
...,...,...,...,...,...,...
75,2585,fazer um bico,&bullet; sexo oral masculino,calão estupidamente carroceiro,fazer um broche; fazer um bobó; mamada; chupa...,5
76,2586,fazer um bobó,&bullet; sexo oral masculino,calão muito carroceiro,fazer um broche; mamada; chupa-me a pichota; ...,4
77,2589,fazer um broche,&bullet; sexo oral masculino,calão estupidamente carroceiro,fazer um bobó; mamada; chupa-me a pichota; fa...,5
78,2631,felácio,&bullet; sexo oral masculino,calão,fazer um broche; fazer um bobó; mamada; chupa...,2


In [52]:
#dicionario[dicionario['description'].str.contains('insulto a árbitro', na=False)]

In [53]:
lyrics_sets = dataset["tokens"]
misogynistic_words = list(dicionario_misoginia["term"])

misogyny_score = pd.Series(dicionario_misoginia.scale.values,index=dicionario_misoginia.term).to_dict()
misogyny_score_data = []
misogynistic_words_by_song = []

for lyrics in lyrics_sets:
  total_misogynistic_score = 0
  misogynistic_words_in_lyrics = [token for token in lyrics if token in misogynistic_words]

  for swear in misogynistic_words_in_lyrics:
      total_misogynistic_score += misogyny_score.get(swear)

  misogynistic_words_by_song.append(misogynistic_words_in_lyrics)
  misogyny_score_data.append(total_misogynistic_score)

dataset["misogynistic_swears"] = misogynistic_words_by_song
dataset["misogynistic_score"] = misogyny_score_data

In [55]:
# correlating misogyny with swearing
dataset['score'].corr(dataset["misogynistic_score"])

0.6534675100752511

In [56]:
#describing swearing and misogyny scores

columns_of_interest = ['score', 'misogynistic_score']
two_columns_of_data = dataset[columns_of_interest]
two_columns_of_data.describe()

,score,misogynistic_score
count,1237.000000,1237.000000
mean,6.884398,1.205335
std,10.988832,5.945943
min,0.000000,0.000000
25%,0.000000,0.000000
50%,4.000000,0.000000
75%,9.000000,0.000000
max,150.000000,150.000000


In [89]:
#describing misogyny per artist

misogyny_score_chart = pd.DataFrame()

misogyny_score_chart["Pontuação média de calão"] = dataset.groupby(["artist_name"])["score"].mean()
misogyny_score_chart["Pontuação mediana de calão"] = dataset.groupby(["artist_name"])["score"].median()
misogyny_score_chart["Pontuação máxima de calão"] = dataset.groupby(["artist_name"])["score"].max()
misogyny_score_chart["Pontuação média de misoginia"] = dataset.groupby(["artist_name"])["misogynistic_score"].mean()
misogyny_score_chart["Pontuação total de misoginia"] = dataset.groupby(["artist_name"])["misogynistic_score"].sum()
misogyny_score_chart["Pontuação máxima de misoginia"] = dataset.groupby(["artist_name"])["misogynistic_score"].max()
#misogyny_score_chart.reset_index(inplace=True)
#misogyny_score_chart.rename(columns={'artist_name': 'Artista'}, inplace = True)

#misogyny_score_chart.to_csv("misogyny_score_chart.csv")
misogyny_score_chart

,Artista,Pontuação média de calão,Pontuação mediana de calão,Pontuação máxima de calão,Pontuação média de misoginia,Pontuação total de misoginia,Pontuação máxima de misoginia
0,5-30,6.000000,5.0,14,3.166667,19,12
1,9 Miller,18.100000,15.5,44,5.800000,58,21
2,Agir,2.037037,0.0,23,0.000000,0,0
3,Alcool Club,6.466667,4.0,30,1.766667,53,20
4,Allen Halloween,12.131579,5.5,88,2.421053,92,19
5,Bispo,9.727273,7.0,72,0.295455,13,3
6,Boss AC,3.750000,2.5,28,0.500000,24,9
7,Carlão (PacMan),4.750000,0.5,50,0.312500,5,2
8,Chico da Tina,12.266667,10.0,53,2.600000,39,15
9,Classe Crua,5.090909,4.0,18,2.181818,24,12


In [81]:
#dataset.to_csv("dataset.csv")


#describing unique words by artist
artist_word_unique = pd.DataFrame()
artist_word_unique["what"] = dataset.groupby(["artist_name", "no_stopwords"])

split_data = [list(i) for i in artist_word_unique["what"]]
artist_word_unique["Artist"] = [i[0] for i in split_data]
artist_word_unique["Lyrics"] = [i[1] for i in artist_word_unique["Artist"]]
artist_word_unique["Artist"] = [i[0] for i in artist_word_unique["Artist"]]
artist_word_unique

,what,Artist,Lyrics
0,"((5-30, ), [artist_name])",5-30,
1,"((5-30, canta som moca baze nova sombra retoca...",5-30,canta som moca baze nova sombra retoca base lá...
2,"((5-30, comenta lamenta quer guarde enquanto l...",5-30,comenta lamenta quer guarde enquanto lentament...
3,"((5-30, quero morrer hoje saudades tanta coisa...",5-30,quero morrer hoje saudades tanta coisa fiz que...
4,"((5-30, tempo chegou hora momento agora testem...",5-30,tempo chegou hora momento agora testemunha nov...
...,...,...,...
1224,"((Xeg, yeh egotripping man contame histórias b...",Xeg,yeh egotripping man contame histórias boy quê ...
1225,"((Xeg, yeh niggas dont chill yeh boy xeg is ch...",Xeg,yeh niggas dont chill yeh boy xeg is chilling ...
1226,"((Xeg, yo mariana andote mancar panorama dread...",Xeg,yo mariana andote mancar panorama dreads andas...
1227,"((Xeg, yo xeg remisturas vol1 bemvindos trabal...",Xeg,yo xeg remisturas vol1 bemvindos trabalho ediç...


In [82]:
vc = artist_word_unique.Artist.value_counts()
vc = vc.reset_index()
vc.rename(columns={'Artist':'Number of Songs', 'index': 'Artist'}, inplace=True)
vc

,Artist,Number of Songs
0,Phoenix RDC,69
1,Sam The Kid,58
2,ProfJam,49
3,Boss AC,47
4,Bispo,44
5,Jimmy P,43
6,Xeg,43
7,Regula,41
8,Dealema,38
9,Allen Halloween,38


In [99]:
#describing unique words by artist

def myfunc(x):
    s = [element for element in x]
    return " ".join(s)

df_perfect = artist_word_unique.groupby("Artist").agg({"Lyrics": lambda x: myfunc(x)})

real_unique = df_perfect.Lyrics.apply(lambda x: len(set([i.strip() for i in x.split(" ") if i != ""])))
real_unique = real_unique.reset_index()
real_unique.rename(columns={'Lyrics':'Palavras únicas'}, inplace=True)
real_unique = real_unique.merge(vc, how="left", on="Artist")
real_unique["ratio_unique"] = real_unique["Palavras únicas"] / real_unique["Number of Songs"]
real_unique.rename(columns={'Artist': 'Artista', "Number of Songs": 'Nº de Canções'}, inplace = True)
final_score = real_unique.merge(misogyny_score_chart, how="left", on="Artista")
final_score["Pontuação Final de Misoginia"] = final_score["Pontuação total de misoginia"] / final_score["Palavras únicas"]
final_score["Pontuação Final de Misoginia/Canção"] = final_score["Pontuação total de misoginia"] / final_score["Nº de Canções"]

In [100]:
final_score.sort_values(by="Pontuação Final de Misoginia/Canção", ascending = False)

,Artista,Palavras únicas,Nº de Canções,ratio_unique,Pontuação média de calão,Pontuação mediana de calão,Pontuação máxima de calão,Pontuação média de misoginia,Pontuação total de misoginia,Pontuação máxima de misoginia,Pontuação Final de Misoginia,Pontuação Final de Misoginia/Canção
14,Deejay Telio,725,20,36.250000,11.750000,0.0,150,7.700000,154,150,0.212414,7.700000
1,9 Miller,1502,10,150.200000,18.100000,15.5,44,5.800000,58,21,0.038615,5.800000
0,5-30,650,6,108.333333,6.000000,5.0,14,3.166667,19,12,0.029231,3.166667
24,Kappa Jotta,2329,20,116.450000,8.650000,4.0,61,3.000000,60,54,0.025762,3.000000
49,Xeg,4769,43,110.906977,8.363636,3.0,113,2.840909,125,81,0.026211,2.906977
8,Chico da Tina,1663,15,110.866667,12.266667,10.0,53,2.600000,39,15,0.023452,2.600000
4,Allen Halloween,3259,38,85.763158,12.131579,5.5,88,2.421053,92,19,0.028230,2.421053
36,ProfJam x benji price,990,6,165.000000,13.833333,11.5,30,2.333333,14,7,0.014141,2.333333
35,ProfJam,4711,49,96.142857,8.408163,6.0,78,2.265306,111,66,0.023562,2.265306
9,Classe Crua,1728,11,157.090909,5.090909,4.0,18,2.181818,24,12,0.013889,2.181818


In [ ]:
artist_word_unique = pd.DataFrame()
artist_word_unique["what"] = dataset.groupby(["artist_name", "no_stopwords"])

split_data = [list(i) for i in artist_word_unique["what"]]
artist_word_unique["Artist"] = [i[0] for i in split_data]
artist_word_unique["Lyrics"] = [i[1] for i in artist_word_unique["Artist"]]
artist_word_unique["Artist"] = [i[0] for i in artist_word_unique["Artist"]]
artist_word_unique